aim: recompute for the BGR box the dS cumsum time series of figure 5 (without the errors previously made)

In [1]:
import os,sys
import numpy as np
# xarray
import xarray as xr

import zarr

In [2]:
from dask.distributed import Client
c = Client()
c

/linkhome/rech/genlop01/uor98hu/.local/lib/python3.9/site-packages/distributed/node.py:183: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46809 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:46809/status,
Dashboard: http://127.0.0.1:46809/status,Workers: 5
Total threads: 20,Total memory: 40.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40061,Workers: 5
Dashboard: http://127.0.0.1:46809/status,Total threads: 20
Started: Just now,Total memory: 40.00 GiB
Comm: tcp://127.0.0.1:36219,Total threads: 4
Dashboard: http://127.0.0.1:36667/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:43739,


In [3]:
%%time
maskBG = xr.open_dataset('/gpfswork/rech/cli/uor98hu/MYDATA/TRANSPORT_WP1/BGRmaskobs.nc', chunks = {"x":200,"y":320}).BGRmask.squeeze()

# ------------------------
# loading data to prepare for the volume weighted mean
# ------------------------
diri="/gpfswork/rech/cli/rcli002/eORCA025.L75/eORCA025.L75-I/"
mesh_hgr=xr.open_dataset(diri+'mesh_hgr.nc', chunks = {"x":200,"y":320}).squeeze()
e1t = mesh_hgr.e1t.fillna(0)
e2t = mesh_hgr.e2t.fillna(0)

bgr_e1t = e1t.where(maskBG,drop=True)
bgr_e2t = e2t.where(maskBG,drop=True)

prefix = "eORCA025.L75-IMHOTEP"
dirww = "/gpfswork/rech/cli/uor98hu/MYDATA/BILAN/BILANS/data4figure/final_after_review/"

# les lignes à modifier entre expériences
# nexpREF = "GAI"
# dirsc = "/gpfsscratch/rech/cli/uor98hu/BILANS/"+nexpREF+"/dSdt/new_comp/bg_data/"

# nexpREF = "AI"
# dirsc = "/gpfsscratch/rech/cli/uor98hu/BILANS/"+nexpREF+"/dSdt/bg_data/"

nexpREF = "S"
dirsc = "/gpfsscratch/rech/cli/uor98hu/BILANS/"+nexpREF+"/dSdt/bg_data/"
# fin lignes à modifier

diridatref="/gpfsstore/rech/cli/rcli002/eORCA025.L75/"+prefix+"."+nexpREF+"-S/"
chunk_size = {"x":200,"y":320, 'deptht':5}

# ------------------------
# read e3t and compute the volume weight
# ------------------------
e3t = xr.open_mfdataset(diridatref+"1m/????/*1m_gridT.nc",decode_coords=False,chunks=chunk_size\
                            ,parallel=True).e3t

bgr_e3t = e3t.chunk({'time_counter':e3t.shape[0]}).where(maskBG,drop=True)

# gridcell volume
btbox = bgr_e3t*bgr_e2t*bgr_e1t
voltotal = btbox.sum(("deptht","y","x"))

CPU times: user 14.9 s, sys: 799 ms, total: 15.7 s
Wall time: 28.7 s


In [4]:
%%time
# ------------------------
# load daily data
# ------------------------
S_1d_bgr =xr.open_mfdataset(dirsc+nexpREF+"_y*_1d_vosaline_BGR.zarr", engine="zarr").vosaline

# select the first day of each month
S_1d_first_day = S_1d_bgr.where(S_1d_bgr.time_counter.dt.day==1, drop=True)

# compute deltaS between S of the first day of two consecutive month
deltaS = (S_1d_first_day.roll(time_counter=-1) - S_1d_first_day) # garder en tete que le dernier point n'est pas bon

# change time coordinates of deltaS to match the one from monthly data
deltaS = deltaS.assign_coords(time_counter=btbox.time_counter)

# compute the volume weighted mean of deltaS in the bGR box
bgr_dSdt_mean = ( (deltaS * btbox).sum(("deptht","y","x")) / voltotal ).compute()

# compute the cumulative in time
bgr_dSdt_mean_comp = bgr_dSdt_mean.cumsum().compute()

# save in the work
bgr_dSdt_mean_comp.to_dataset(name='dScumsum').to_netcdf(path = dirww+nexpREF+"_dScumsum_1980_2018_after_review.nc", mode='w')

CPU times: user 34.2 s, sys: 1.83 s, total: 36.1 s
Wall time: 2min


In [73]:
# new_Gtotal = xr.open_dataset(dirww+nexpREF+"_dScumsum_1980_2018_after_review.nc").dScumsum